# **Installing Requirements**

Since the project is developed by different people, we will install all the requirements using the requirements.txt file which specifies all the packets' version that must be installed.

In [ ]:
%pip install -r ../requirements.txt

# **Downloading Files from GDrive**

# **Face Recognition**

Face recognition is a computer vision task that involves identifying and verifying a person's identity based on their facial features. This process can be broken down into these steps:

1. **Detection**: Identifying faces in images or video frames.
2. **Feature** Extraction: Capturing unique facial characteristics.
3. **Representation**: Creating a distinctive template for each face.
4. **Model Training**: Associating templates with known identities during training.
5. **Matching**: Comparing a new face's template to stored ones for identification.
6. **Decision**: Determining a match based on a similarity threshold.

Nowadays, these steps are performed through deep learning models. In the following section we will provide a simple implementation through a pre-trained model and our paper implementation (further details in the next sections).

## **Pre-trained Model**

This model is provided by the dlib library. Dlib's face recognition model is based on classical machine learning techniques rather than deep learning. It uses a combination of HOG (Histogram of Oriented Gradients) features and a SVM (Support Vector Machine) classifier to identify and recognize faces.

### **Load all the user faces**

Now we will define functions to detect faces, extract facial landmarks, and compute facial encodings. We will save the usefull information into a dict that represents the known users.

In [ ]:
import os
import cv2
import dlib
import numpy as np

def face_rects(image, face_detector):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Convert the image to grayscale
    rects = face_detector(gray, 1) # Detect faces in the grayscale image

    return rects


def face_landmarks(image, shape_predictor, face_detector):
    # Compute the face landmarks
    return [shape_predictor(image, face_rect) for face_rect in face_rects(image, face_detector)]


def face_encodings(image, face_encoder, shape_predictor, face_detector):
    # Compute the facial embeddings for each face (128-d vector that describes the face in an image)
    return [np.array(face_encoder.compute_face_descriptor(image, face_landmark)) for face_landmark in face_landmarks(image, shape_predictor, face_detector)]




face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor("../Models/shape_predictor_68_face_landmarks.dat")
face_encoder = dlib.face_recognition_model_v1("../Models/dlib_face_recognition_resnet_model_v1.dat")

known_faces = {} # Dict that will store the user's embeddings

base_directory = "../UserFaces/" # Directory containing user faces

# Iterate through directories
for user_name in os.listdir(base_directory):

    user_path = os.path.join(base_directory, user_name)

    # Iterate through face images in each user directory
    for filename in os.listdir(user_path):
        image_path = os.path.join(user_path, filename)

        img = cv2.imread(image_path) # Read the image
        new_encodings = face_encodings(img, face_encoder, shape_predictor, face_detector) # Get the embeddings

        encodings = known_faces.get(user_name, []) 
        encodings.extend(new_encodings) # Add the embeddings to the already saved ones
        known_faces[user_name] = encodings


print(known_faces.keys())

### **User Recognition**

In this part we will use the obtained information perform a real-time recognition through the webcam.

In [ ]:
def nb_of_matches(known_encodings, unknown_encoding):
    # compute the Euclidean distance between the current face encoding 
    # and all the face encodings in the database
    distances = np.linalg.norm(known_encodings - unknown_encoding, axis=1)
    # keep only the distances that are less than the threshold
    small_distances = distances <= 0.6
    # return the number of matches
    return sum(small_distances)


cap = cv2.VideoCapture(0) # Open a connection to the webcam (0 represents the default camera)

while True:
    ret, frame = cap.read() # Read a frame from the webcam

    frame_encodings = face_encodings(frame, face_detector=face_detector, face_encoder=face_encoder, shape_predictor=shape_predictor)
    names = []

    for encoding in frame_encodings:
        counts = {}

        for (name, known_encodings) in known_faces.items():
            counts[name] = nb_of_matches(known_encodings, encoding)
        
        if all(count == 0 for count in counts.values()):
            name = "Unknown"
        else:
            name = max(counts, key=counts.get)

        names.append(name)

    for rect, name in zip(face_rects(frame, face_detector), names):
        # get the bounding box for each face using the `rect` variable
        x1, y1, x2, y2 = rect.left(), rect.top(), rect.right(), rect.bottom()
        # draw the bounding box of the face along with the name of the person
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, name, (x1, y1 - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    # Display the result
    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): # Break the loop if the 'q' key is pressed
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()
